# Project 4: West Nile Virus Prediction
____________

# Part 4 - Kaggle Predictions

This notebook is just for predicting the Wnv presence on the Kaggle dataset.

### Group 4, SG-DSI-18

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier

### Import data

In [2]:
# Import train data to see which features are required
X_train = pickle.load(open('../datasets/X_train.pkl', 'rb'))

# Import weather data to merge with test data
weather = pickle.load(open('../datasets/weather_final.pkl', 'rb'))

In [3]:
weather.head()

,Station,Date,WeekTavg,WeekWetBulb,WeekWetDry,WeekPrecipTotal,WeekStnPressure,WeekSeaLevel,WeekResultSpeed,WeekResultDir,...,windlag3,windlag4,sunlag1,sunlag2,sunlag3,sunlag4,humidlag1,humidlag2,humidlag3,humidlag4
0,1,2007-05-01,60.333333,50.0,0.166667,0.000833,29.42,30.071667,10.666667,9.5,...,10.1,9.1,14.111111,13.111111,12.111111,11.111111,45.383333,44.383333,43.383333,42.383333
1,2,2007-05-01,60.333333,50.0,0.166667,0.000833,29.42,30.071667,10.666667,9.5,...,10.1,9.1,14.111111,13.111111,12.111111,11.111111,45.383333,44.383333,43.383333,42.383333
2,1,2007-05-02,60.333333,50.0,0.166667,0.000833,29.42,30.071667,10.666667,9.5,...,10.1,9.1,14.111111,13.111111,12.111111,11.111111,45.383333,44.383333,43.383333,42.383333
3,2,2007-05-02,60.333333,50.0,0.166667,0.000833,29.42,30.071667,10.666667,9.5,...,10.1,9.1,14.111111,13.111111,12.111111,11.111111,45.383333,44.383333,43.383333,42.383333
4,1,2007-05-03,60.333333,50.0,0.166667,0.000833,29.42,30.071667,10.666667,9.5,...,10.1,9.1,14.111111,13.111111,12.111111,11.111111,45.383333,44.383333,43.383333,42.383333


In [4]:
# Import Kaggle Test data
test = pd.read_csv('../datasets/test.csv')

# Import sample for submission
sample_submission = pd.read_csv('../datasets/sampleSubmission.csv')

In [5]:
test.head(20)

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9
5,6,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TARSALIS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9
6,7,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",UNSPECIFIED CULEX,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9
7,8,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX ERRATICUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9
8,9,2008-06-11,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX PIPIENS/RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9
9,10,2008-06-11,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9


In [6]:
test.shape

(116293, 11)

In [7]:
# Convert date to datetime format
test['Date'] = pd.to_datetime(test['Date'])

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Id                      116293 non-null  int64         
 1   Date                    116293 non-null  datetime64[ns]
 2   Address                 116293 non-null  object        
 3   Species                 116293 non-null  object        
 4   Block                   116293 non-null  int64         
 5   Street                  116293 non-null  object        
 6   Trap                    116293 non-null  object        
 7   AddressNumberAndStreet  116293 non-null  object        
 8   Latitude                116293 non-null  float64       
 9   Longitude               116293 non-null  float64       
 10  AddressAccuracy         116293 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 9.8+ MB


We see that the test data is complete with no missing/null values. The data types are also correct.

In [9]:
sample_submission.head()

,Id,WnvPresent
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


We want to obtain a `.csv` file with just the Id and the predictions to be used for submission.

### Check for duplicates

In [10]:
test[test.duplicated()]

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy


Test data is clean with no duplicates.

## Transformation of Test data

The test data which we have only contain the mosquito trap information, we want to add in the weather data and also transform the features to match that of the train data so that we can send it into our production model for predictions.

In [11]:
# As we will be using lat and long for location, remove other location variables
test.drop(columns=['Address','Block','Street','AddressNumberAndStreet', 'AddressAccuracy'], inplace=True)

In [12]:
# Map the species
test['species_no'] = test['Species'].map({'CULEX PIPIENS/RESTUANS': 2,'CULEX PIPIENS': 2,'CULEX RESTUANS': 1})
test['species_no'].fillna(value=0,inplace=True)

# Drop the Species
test.drop(columns=['Species'], inplace=True)

In [13]:
# Check Species after update
test.head()

,Id,Date,Trap,Latitude,Longitude,species_no
0,1,2008-06-11,T002,41.95469,-87.800991,2.0
1,2,2008-06-11,T002,41.95469,-87.800991,1.0
2,3,2008-06-11,T002,41.95469,-87.800991,2.0
3,4,2008-06-11,T002,41.95469,-87.800991,0.0
4,5,2008-06-11,T002,41.95469,-87.800991,0.0


In [14]:
# One-hot encode the traps
test = pd.get_dummies(data=test, columns=['Trap'])

In [15]:
# Add in column to indicate the weather station for each trap in test data
import geopy.distance

station1 = (41.995, -87.933) #Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT
station2 = (41.786, -87.752) #Station 2: CHICAGO MIDWAY INTL ARPT
def get_station(lat, long):
    dist1 = geopy.distance.distance((lat, long), station1).km
    dist2 = geopy.distance.distance((lat, long), station2).km
    return 1 if dist1 < dist2 else 2

test['Station'] = [get_station(test.loc[i,'Latitude'],test.loc[i,'Longitude']) for i in test.index]

In [16]:
# Merge weather data with test data
test = pd.merge(test, weather, on=['Date','Station'], how='left')

In [17]:
# Add column for trap_sprayed to be zero since we do not have data of spraying effort in the years of test data
test['trap_sprayed'] = 0

In [18]:
# Check data after update
test.head()

,Id,Date,Latitude,Longitude,species_no,Trap_T001,Trap_T002,Trap_T002A,Trap_T002B,Trap_T003,...,windlag4,sunlag1,sunlag2,sunlag3,sunlag4,humidlag1,humidlag2,humidlag3,humidlag4,trap_sprayed
0,1,2008-06-11,41.95469,-87.800991,2.0,0,1,0,0,0,...,9.971429,15.090476,14.959524,14.783333,14.566667,72.228571,60.714286,52.771429,65.485714,0
1,2,2008-06-11,41.95469,-87.800991,1.0,0,1,0,0,0,...,9.971429,15.090476,14.959524,14.783333,14.566667,72.228571,60.714286,52.771429,65.485714,0
2,3,2008-06-11,41.95469,-87.800991,2.0,0,1,0,0,0,...,9.971429,15.090476,14.959524,14.783333,14.566667,72.228571,60.714286,52.771429,65.485714,0
3,4,2008-06-11,41.95469,-87.800991,0.0,0,1,0,0,0,...,9.971429,15.090476,14.959524,14.783333,14.566667,72.228571,60.714286,52.771429,65.485714,0
4,5,2008-06-11,41.95469,-87.800991,0.0,0,1,0,0,0,...,9.971429,15.090476,14.959524,14.783333,14.566667,72.228571,60.714286,52.771429,65.485714,0


In [19]:
# Extract only the features where the train data uses
features = X_train.columns
features

Index(['species_no', 'trap_sprayed', 'templag1', 'templag2', 'templag3',
       'templag4', 'wetbulblag1', 'wetbulblag2', 'wetbulblag3', 'wetbulblag4',
       ...
       'Trap_T230', 'Trap_T231', 'Trap_T232', 'Trap_T233', 'Trap_T235',
       'Trap_T236', 'Trap_T237', 'Trap_T238', 'Trap_T900', 'Trap_T903'],
      dtype='object', length=182)

In [20]:
X_test = test[features]
X_test.head()

,species_no,trap_sprayed,templag1,templag2,templag3,templag4,wetbulblag1,wetbulblag2,wetbulblag3,wetbulblag4,...,Trap_T230,Trap_T231,Trap_T232,Trap_T233,Trap_T235,Trap_T236,Trap_T237,Trap_T238,Trap_T900,Trap_T903
0,2.0,0,74.285714,61.428571,55.714286,54.142857,67.571429,54.285714,47.571429,48.571429,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0,74.285714,61.428571,55.714286,54.142857,67.571429,54.285714,47.571429,48.571429,...,0,0,0,0,0,0,0,0,0,0
2,2.0,0,74.285714,61.428571,55.714286,54.142857,67.571429,54.285714,47.571429,48.571429,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0,74.285714,61.428571,55.714286,54.142857,67.571429,54.285714,47.571429,48.571429,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0,74.285714,61.428571,55.714286,54.142857,67.571429,54.285714,47.571429,48.571429,...,0,0,0,0,0,0,0,0,0,0


## Predictions

Now that the test data is ready, we can load in our production model and use it for predictions.

In [21]:
# Load in production model and scaler
ss = pickle.load(open('scaler.pkl', 'rb'))
model = pickle.load(open('production_model.pkl', 'rb'))

print(ss)
print(model)

StandardScaler()
AdaBoostClassifier(learning_rate=1, n_estimators=20, random_state=42)


In [22]:
# Scale test data
Xs_test = ss.transform(X_test)

In [23]:
predictions = model.predict(Xs_test)
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [24]:
# Set up dataframe to save as .csv for submission
submission = pd.DataFrame(test['Id'])

# Add in the predictions
submission['WnvPresent'] = predictions

In [25]:
submission.head(10)

,Id,WnvPresent
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0
6,7,0
7,8,0
8,9,0
9,10,0


In [26]:
submission['WnvPresent'].value_counts()

0    104622
1     11671
Name: WnvPresent, dtype: int64

#### Save predictions as `.csv` and submit

In [27]:
# Save csv
submission.to_csv('../datasets/submission_predictions.csv', index=False)

![Submission](../images/kaggle_submission.png)